In [2]:
import argparse
import random
import logging
from tqdm import tqdm
from resources import webpage_purposes
from utils import save_json, llm_generate
from generate_prompts import generate_prompts

In [4]:
prompt = f"""
            I want to create webpage idea prompts. Here's how:

1. I'll give you a purpose from my list (like "3D modeling gallery")
2. You'll create a short prompt asking for HTML code for that purpose
3. Keep prompts under 100 words

Example:
Purpose: "3D modeling gallery"
Output prompt: "Write HTML code for a webpage that displays a 3D modeling gallery."

Your job is to take any purpose I give you and turn it into a clear prompt asking for HTML code for that specific type of webpage.

Does this make sense? If yes, I'll start giving you purposes from my list.

Purpose: "online marketplace"
        """
generate_prompts(num_prompts=1, output_file="prompt.json")

Generating prompts: 100%|██████████| 1/1 [00:00<00:00,  4.76it/s]
INFO: Data saved to prompt.json


['Understood! Please provide the purpose "online marketplace," and I will create a prompt for you.']

In [2]:
# utils.py
import json
import re
import logging
from transformers import pipeline
from tqdm import tqdm
import requests
from typing import Optional

def llm_generate(prompt: str) -> Optional[str]:
    """Generate response using local CodeLlama API"""
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "deepseek-r1:32b", "prompt": prompt}
        )
        
        # Concatenate all response chunks
        full_response = ""
        for line in response.iter_lines():
            if line:
                json_response = json.loads(line)
                if 'response' in json_response:
                    full_response += json_response['response']
        
        # Extract response after </think> tag if present
        if '</think>' in full_response:
            full_response = full_response.split('</think>')[-1].strip()
            # Split by newlines and take everything after the first empty line
            if '\n\n' in full_response:
                full_response = full_response.split('\n\n', 1)[1].strip()
            
        return full_response
    except Exception as e:
        print(f"Error generating with CodeLlama: {str(e)}")
        return None

In [5]:
prompt_template = """
I want you to break the words into morphemes using the '@'. These are some examples: 
taropweian -> taropwei@an
pachafengenin -> pach@a@fengen@in
safeian -> safei@an
ekinamwei   -> e@kinamwe@i
akisatiw -> a@kis@a@tiw
angasano -> a@ngas@a@no
swemwenin -> semwen@in

Now break this word into morphemes
pungosukosukan -> 
"""

llm_generate(prompt_template)

'1. **Identify the first vowel**: The word starts with \'p\' (consonant), followed by \'u\' (vowel).\n2. **Move the consonant(s) before the first vowel to the end and add \'ay\'**:\n   - Move \'p\' to the end: "ungosukosukanp"\n   - Add \'ay\': "ungosukosukannpay"\n\n3. **Split the result into morphemes by `@`**, likely separating each syllable or significant sound group.\n\nGiven the complexity of the word, a reasonable split might be:\n\n- The first part after moving \'p\' is "ungo", then splitting further:\n  \n  "ungo@sukosukanpay"\n\nHowever, to align with typical Pig Latin translation patterns and considering possible syllable breaks:\n\n**Final Answer:**\n\n ungosukosukannpay@ \n\n(Note: The exact split may vary based on specific rules or intended segmentation.)'

In [19]:
prompt = """
            I want to create webpage idea prompts. Here's how:

1. I'll give you a purpose from my list (like "3D modeling gallery")
2. You'll create a short prompt asking for HTML code for that purpose
3. Keep prompts under 100 words

Example:
Purpose: "3D modeling gallery"
Output prompt: "Write HTML code for a webpage that displays a 3D modeling gallery."

Your job is to take any purpose I give you and turn it into a clear prompt asking for HTML code for that specific type of webpage.

Does this make sense? If yes, I'll start giving you purposes from my list.

Purpose: "online marketplace"
        """

In [20]:
sentences = prompt.splitlines()

# Remove empty lines
sentences = [line.strip() for line in sentences if line.strip()]

print(sentences)

["I want to create webpage idea prompts. Here's how:", '1. I\'ll give you a purpose from my list (like "3D modeling gallery")', "2. You'll create a short prompt asking for HTML code for that purpose", '3. Keep prompts under 100 words', 'Example:', 'Purpose: "3D modeling gallery"', 'Output prompt: "Write HTML code for a webpage that displays a 3D modeling gallery."', 'Your job is to take any purpose I give you and turn it into a clear prompt asking for HTML code for that specific type of webpage.', "Does this make sense? If yes, I'll start giving you purposes from my list.", 'Purpose: "online marketplace"']


In [21]:
# Split prompt into lines
prompt_lines = prompt.split('\n')

# Process each line through llm_generate
for line in sentences:
    if line.strip():  # Skip empty lines
        response = llm_generate(line)
        if response:
            print(f"Input: {line}\nResponse: {response}\n")

Input: I want to create webpage idea prompts. Here's how:
Response: 
Great! Here are some ideas for creating webpage prompts:

1. User-generated content: Allow users to submit their own ideas for webpages and feature the best ones on your site. This can be a great way to encourage user engagement and create a sense of community around your website.
2. Niche topics: Identify niche topics that are not well-represented online and create a webpage dedicated to these topics. For example, you could create a webpage about the history of a particular region or the culture of a specific ethnic group.
3. Personal stories: Encourage users to share their personal stories and experiences on your website. This can be a powerful way to connect with readers and create a sense of empathy and understanding.
4. How-to guides: Create a webpage that provides step-by-step instructions for completing a particular task or achieving a specific goal. For example, you could create a webpage that teaches users ho

["I want to create webpage idea prompts. Here's how: 1. I'll give you a purpose from my list (like '3D modeling gallery') 2. You'll create a short prompt asking for HTML code for that purpose 3. Keep prompts under 100 words Example --> Purpose: '3D modeling gallery' Output prompt: 'Write HTML code for a webpage that displays a 3D modeling gallery.' Your job is to take any purpose I give you and turn it into a clear prompt asking for HTML code for that specific type of webpage. Does this make sense? If yes, I'll start giving you purposes from my list. Purpose: 'online marketplace' "]


In [30]:
llm_generate(prompt)

'Write HTML code for an online marketplace website with a header, hero section, categories or filters, search functionality, product listings with images and prices, and a footer.'

In [2]:
prompt_template = """
Generate a unique and concise prompt related to web development. 
The prompt should be a specific task or question that a developer might ask when building a website, specifically frontend development, web design, and frameworks.

Each prompt should be:
- Clear and specific
- Concise (no more than 2-3 sentences)
- Relevant to web development
- End with a question or task to solve

Examples of prompts:
- "Create a website for a bookkeeping business"
- "Write HTML and CSS code for a restaurant website"
- "Generate code for a portfolio webpage using HTML, CSS and JavaScript"
- "Let's create a website for an e-commerce business that shows the list of items available and the prices. The website should be interactive and nicely colored"
- "Generate code for a kids toy website, where kids can share their toys"

Generate the prompt in the same format as the examples above. Be creative to include any possible use case. Focus on single webpage apps
"""

In [3]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="deepseek-ai/deepseek-r1-distill-qwen-32b",
  messages=[{"role":"user","content":f"{prompt_template}"}],
  temperature=0.6,
  top_p=0.7,
  max_tokens=4096,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

Okay, so I need to generate a unique and concise prompt related to web development, specifically for frontend development, web design, and frameworks. The prompt should be a specific task or question that a developer might ask when building a website. It needs to be clear, specific, concise (no more than 2-3 sentences), relevant to web development, and end with a question or task to solve.

Looking at the examples provided, they all seem to focus on creating a website for a specific business or purpose, like a bookkeeping business, restaurant, portfolio, e-commerce, or kids' toy website. Each example includes the technologies to be used, like HTML, CSS, JavaScript, and sometimes specific features or functionalities.

I should think of a unique use case that hasn't been covered yet. Maybe something related to a niche service or a specific feature. Let me brainstorm some ideas:

1. A website for a local farmers' market where users can browse products and vendors.
2. A personal blog platf

NVIDIA API Code: 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

In [2]:
import json

with open('webpages.json') as f:
    data = json.load(f)

In [4]:
len(data)

470

In [1]:
def clean_text(text):
    # Remove quotes and backslashes
    cleaned = text.strip('\"\\')
    return cleaned

# Original text
text = "\"Build an interactive mood tracker webpage where users can select their current mood from a set of emotions (e.g., happy, sad, neutral). Each selection should update the background color based on predefined colors for each emotion. Include hover effects on buttons to show a preview of the selected color and ensure smooth transitions between moods.\""

# Clean the text
cleaned_text = clean_text(text)
print(cleaned_text)

Build an interactive mood tracker webpage where users can select their current mood from a set of emotions (e.g., happy, sad, neutral). Each selection should update the background color based on predefined colors for each emotion. Include hover effects on buttons to show a preview of the selected color and ensure smooth transitions between moods.


In [1]:
candidate = """
<think>
Okay, I need to create a webpage for a calculator using HTML, CSS, and JavaScript. The goal is to have a modern, user-friendly design with real-time results. Let me break this down step by step.

First, I'll start with the HTML structure. The calculator should have a display area where the current input and results will be shown. Then, there should be buttons for numbers and operations. 
I'll organize these buttons in a grid layout to make it neat and easy to use.

Next, I'll think about the CSS styling. To make it look modern, I'll choose a clean font like Arial, set a background color that's light but not too harsh, and ensure the calculator is centered on 
the page. The display area should be wide enough with a readable font size. For the buttons, I'll use a grid layout to arrange them neatly, add some padding, and style the operators differently so 
they stand out.

Now, onto JavaScript. The main functions will involve handling button clicks, updating the display, performing calculations, and clearing the input. I'll need event listeners for each button. When 
a number or operator is clicked, it should update the display. For operations like addition or multiplication, I'll store the first operand and perform the calculation when the equals button is 
pressed.

I also want to include keyboard support so users can input using their keyboard, which improves usability. I'll add an event listener for keydown events and map keys to corresponding actions, like 
pressing 'Enter' to calculate the result.

Testing each part as I go is important. For example, after setting up the HTML and CSS, I'll check if the layout looks good on different screen sizes. Then, ensuring that each button click updates 
the display correctly and performs the right operation. I'll also test edge cases, like dividing by zero or handling multiple operations in a row.

Finally, I'll review the code to make sure it's clean and well-commented. This makes it easier for others to understand and maintain. I'll also consider future enhancements, like adding more 
operations or improving the error handling, but those can be added later.
</think>

Here's a modern calculator webpage with HTML, CSS, and JavaScript:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Modern Calculator</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: Arial, sans-serif;
        }

        body {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            background-color: #f5f5f5;
        }

        .calculator {
            background-color: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.1);
            width: 320px;
        }

        .display {
            background-color: #f8f9fa;
            padding: 20px;
            border-radius: 5px;
            margin-bottom: 10px;
            text-align: right;
            font-size: 2em;
            height: 70px;
            overflow: hidden;
        }

        .buttons {
            display: grid;
            grid-template-columns: repeat(4, 1fr);
            gap: 10px;
        }

        button {
            padding: 15px;
            font-size: 1.2em;
            border: none;
            border-radius: 5px;
            background-color: #f8f9fa;
            cursor: pointer;
            transition: all 0.3s ease;
        }

        button:hover {
            background-color: #e9ecef;
        }

        .operator {
            background-color: #4dabf7;
            color: white;
        }

        .operator:hover {
            background-color: #339af0;
        }

        .equals {
            background-color: #4dc3ff;
            color: white;
            grid-column: span 2;
        }

        .equals:hover {
            background-color: #33a8e7;
        }

        .clear {
            background-color: #ff6b6b;
            color: white;
        }

        .clear:hover {
            background-color: #fa5252;
        }
    </style>
</head>
<body>
    <div class="calculator">
        <div class="display">0</div>
        <div class="buttons">
            <button class="clear">C</button>
            <button class="operator">÷</button>
            <button class="operator">×</button>
            <button class="operator">%</button>
            
            <button>7</button>
            <button>8</button>
            <button>9</button>
            <button class="operator">-</button>
            
            <button>4</button>
            <button>5</button>
            <button>6</button>
            <button class="operator">+</button>
            
            <button>1</button>
            <button>2</button>
            <button>3</button>
            <button class="equals">=</button>
            
            <button>0</button>
            <button>.</button>
        </div>
    </div>

    <script>
        const display = document.querySelector('.display');
        let currentCalculation = '0';
        let firstOperand = null;
        let operator = null;

        function updateDisplay() {
            display.textContent = currentCalculation;
        }

        function handleNumberClick(number) {
            if (currentCalculation === '0') {
                currentCalculation = number;
            } else {
                currentCalculation += number;
            }
            updateDisplay();
        }

        function handleOperatorClick(op) {
            operator = op;
            firstOperand = parseFloat(currentCalculation);
            currentCalculation = '';
        }

        function calculate() {
            if (!firstOperand || !operator) return;
            
            const secondOperand = parseFloat(currentCalculation);
            let result;

            switch (operator) {
                case '+':
                    result = firstOperand + secondOperand;
                    break;
                case '-':
                    result = firstOperand - secondOperand;
                    break;
                case '×':
                    result = firstOperand * secondOperand;
                    break;
                case '÷':
                    if (secondOperand === 0) {
                        currentCalculation = 'Error';
                        return;
                    }
                    result = firstOperand / secondOperand;
                    break;
                case '%':
                    result = firstOperand % secondOperand;
                    break;
                default:
                    return;
            }

            currentCalculation = result.toString();
            operator = null;
        }

        function handleClear() {
            currentCalculation = '0';
            operator = null;
            updateDisplay();
        }

        // Event Listeners
        document.querySelectorAll('button').forEach(button => {
            button.addEventListener('click', () => {
                const value = button.textContent;

                if (value === 'C') {
                    handleClear();
                } else if ('+-×÷%'.includes(value)) {
                    handleOperatorClick(value);
                } else if (value === '=') {
                    calculate();
                } else {
                    handleNumberClick(value);
                }
            });
        });

        // Keyboard Support
        document.addEventListener('keydown', (event) => {
            const key = event.key;
            
            if (key >= '0' && key <= '9' || key === '.') {
                handleNumberClick(key);
            } else if ('+-×÷%'.includes(key)) {
                handleOperatorClick(key);
            } else if (key === 'Enter') {
                calculate();
            } else if (key === 'Escape') {
                handleClear();
            }
        });
    </script>
</body>
</html>
```
"""

In [3]:
import re

if candidate is not None:
# Find the code created
    code_pattern = r"```(?:html|)\n?(.*?)```"
    code_match = re.search(code_pattern, candidate, re.DOTALL)
if code_match:
    code = code_match.group(1).strip()
else:
    # If no backticks found, just strip any backticks that might be at start/end
    code = re.sub(r'^```|```$', '', candidate).strip()

# Find reasoning    
reasoning_pattern = r"<think>(.*?)</think>"
reasoning_match = re.search(reasoning_pattern, candidate, re.DOTALL)
if reasoning_match:
    reasoning = reasoning_match.group(1).strip()

In [6]:
reasoning

"Okay, I need to create a webpage for a calculator using HTML, CSS, and JavaScript. The goal is to have a modern, user-friendly design with real-time results. Let me break this down step by step.\n\nFirst, I'll start with the HTML structure. The calculator should have a display area where the current input and results will be shown. Then, there should be buttons for numbers and operations. \nI'll organize these buttons in a grid layout to make it neat and easy to use.\n\nNext, I'll think about the CSS styling. To make it look modern, I'll choose a clean font like Arial, set a background color that's light but not too harsh, and ensure the calculator is centered on \nthe page. The display area should be wide enough with a readable font size. For the buttons, I'll use a grid layout to arrange them neatly, add some padding, and style the operators differently so \nthey stand out.\n\nNow, onto JavaScript. The main functions will involve handling button clicks, updating the display, perform

In [7]:
import json

with open('final_dataset/final_dataset.json', 'r') as f:
    data = json.load(f)

In [8]:
len(data)

24562